In [6]:
username = "Alex"
pathtorepo = "C:\\Users\\" *username *  "\\Desktop\\"
using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")
include(pathtorepo * "dynamical-systems\\system.jl")


using StaticArrays, DifferentialEquations, DynamicalSystems, JLD
E, x, y  = -40..40, -5..5,  -5..5
box = E × x × y
using GLMakie, LinearAlgebra

  Activating project at `C:\Users\Alex\Desktop\dynamical-systems\env\integrate`


In [7]:
cd("C:\\Users\\" *username *  "\\Desktop\\dynamical-systems\\Tsodyks Markram\\Levanova\\3 набор параметров\\Сопоставление с матконт\\файлы matlab")
I0_hom = load("I0_hom_hom.jld")["data"]
u0_hom = load("U0_hom_hom.jld")["data"]
I0_hom = I0_hom[:]
u0_hom = u0_hom[:];

In [8]:
const τ = 0.013;  const τD = 0.07993;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4; const α = 1.58; const ΔU0 = 0.305

0.305

In [9]:
index_hom = 4
I0 = I0_hom[index_hom]; U0 = u0_hom[index_hom]
p = SA[α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0]
u0 = [11.325905642223786, 0.6594706953104683, 0.4863175026548461];

In [10]:
ds = CoupledODEs(TM, u0, p)
fp, _, _ = fixedpoints(ds, box, jacob_TM_)
ϵ_sep = 1e-6
Ju0 = jacob_TM_(fp[1], p, 0)

eigen_val_vec = eigen(Ju0)
eigen_vectors = eigen_val_vec.vectors
v1 = real(eigen_vectors[:, 1])
dot_on_sep_stable = fp[1] + v1*ϵ_sep

3-element SVector{3, Float64} with indices SOneTo(3):
 8.345774611099296
 0.7384945673457723
 0.4382983362268263

In [51]:
t = 1000.0; tspan = [0.0, t]
prob = ODEProblem(TM, dot_on_sep_stable, tspan, p)
sol = solve(prob, alg = Vern9(), adaptive = true, abstol = 1e-14, reltol = 1e-14)

retcode: Success
Interpolation: specialized 9th order lazy interpolation
t: 88258-element Vector{Float64}:
    0.0
    0.06966748897871437
    0.11962903077235723
    0.1724198104878905
    0.22122741796521908
    0.2669474693934435
    0.31170858235802057
    0.3538543811142312
    0.39668502624108987
    0.4459395221184082
    ⋮
  999.9410597705648
  999.9498132578212
  999.9574765447969
  999.9657956130449
  999.9735219992228
  999.9830762832659
  999.992229358465
  999.9999509734139
 1000.0
u: 88258-element Vector{SVector{3, Float64}}:
 [8.345774611099296, 0.7384945673457723, 0.4382983362268263]
 [8.345774252229997, 0.7384945849485874, 0.4382983319465129]
 [8.345774077330837, 0.7384945935275391, 0.4382983298604495]
 [8.345773944184858, 0.7384946000584787, 0.43829832827238113]
 [8.345773855272217, 0.7384946044197412, 0.43829832721189116]
 [8.34577379372831, 0.7384946074385392, 0.43829832647783445]
 [8.345773748610197, 0.7384946096516245, 0.43829832593969503]
 [8.345773716441304, 0.7

In [38]:
"""
homcurve = range(1, 150, step = 1)
for index in homcurve
    
    I0 = I0_hom[index]; U0 = u0_hom[index]
    p = SA[α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0]

    println("I0: $(I0); U0: $(U0)")

    ds = CoupledODEs(TM, u0, p)
    fp, _, _ = fixedpoints(ds, box, jacob_TM_)
    ϵ_sep = 1e-6
    Ju0 = jacob_TM_(fp[1], p, 0)

    eigen_val_vec = eigen(Ju0)
    eigen_vectors = eigen_val_vec.vectors
    v1 = real(eigen_vectors[:, 1])
    u0 = fp[1] + v1*ϵ_sep

    prob = ODEProblem(TM, u0, tspanback, p)
    sol = solve(prob, Rodas5P(), adaptive = true, abstol = 1e-9, reltol = 1e-9, verbose = false)
    println("Index: $(sol.retcode)")    
    println("----------------------")
end"""

"homcurve = range(1, 150, step = 1)\nfor index in homcurve\n    \n    I0 = I0_hom[index]; U0 = u0_hom[index]\n    p = SA[α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0]\n\n    println(\"I0: -1.706333384363191; U0: 0.26500275817125657\")\n\n    ds = CoupledODEs(TM, u0, p)\n    fp, _, _ " ⋯ 167 bytes ⋯ " real(eigen_vectors[:, 1])\n    u0 = fp[1] + v1*ϵ_sep\n\n    prob = ODEProblem(TM, u0, tspanback, p)\n    sol = solve(prob, Rodas5P(), adaptive = true, abstol = 1e-9, reltol = 1e-9, verbose = false)\n    println(\"Index: Success\")    \n    println(\"----------------------\")\nend"

In [54]:
ts, tf = 1, 20000

idx, idy, idz = 2, 3, 1
dis  = 60;
    
f = Figure(resolution = (700, 700))
ax = LScene(f[1, 1], show_axis = true)
scale!(ax.scene, 50, 50, 1)

lines!(sol[idx, ts:tf], sol[idy, ts:tf], sol[idz, ts:tf], linewidth = 1.0, color = :deeppink)

for (i, ep) in enumerate(fp)
    scatter!(ep[idx], ep[idy], ep[idz], color = :blue, markersize = 8)
end

display(GLMakie.Screen(), f)

GLMakie.Screen(...)